In [1]:
%pip install nltk konlpy beautifulsoup4 requests pandas transformers torch --user

In [6]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import re
import pandas as pd
import csv
from collections import Counter
import torch
from transformers import BertTokenizer, BertModel

In [ ]:
# 코스피, 코스닥 종목명 및 코드 추출

def extract_stocks_name():
    df_kospi = pd.read_excel('kospi_code.xlsx', dtype={'단축코드': str})
    df_kosdaq = pd.read_excel('kosdaq_code.xlsx', dtype={'단축코드': str})

    filterd_df_kospi = df_kospi[df_kospi['그룹코드'] == 'ST']
    filterd_df_kosdaq = df_kosdaq[df_kosdaq['증권그룹구분코드'] == 'ST']

    result_df_kospi = filterd_df_kospi[['한글명', '단축코드']].copy()
    result_df_kosdaq = filterd_df_kosdaq[['한글종목명', '단축코드']].copy()
    
    
    result_df_kospi.rename(columns={'한글명': '종목명'}, inplace=True)
    result_df_kosdaq.rename(columns={'한글종목명': '종목명'}, inplace=True)

    result_df_kospi['단축코드'] = result_df_kospi['단축코드'].apply(lambda x: x.zfill(6))
    result_df_kosdaq['단축코드'] = result_df_kosdaq['단축코드'].apply(lambda x: x.zfill(6))

    result_df_kospi.to_csv('../data/raw_data/kospi_stocks.csv', index=False, encoding='utf-8-sig', quoting=csv.QUOTE_ALL)
    result_df_kosdaq.to_csv('../data/raw_data/kosdaq_stocks.csv', index=False, encoding='utf-8-sig')
    print("CSV 파일이 성공적으로 저장되었습니다: kospi_stocks.csv, kosdaq_stocks.csv")

    #df = pd.read_csv('kosdaq_stocks.csv')
    #print(df['단축코드'].head(10))
    
extract_stocks_name()

CSV 파일이 성공적으로 저장되었습니다: kospi_stocks.csv, kosdaq_stocks.csv
0    000250
1    000440
2    001000
3    001540
4    001810
5    001840
6    002230
7    002290
8    002680
9    002800
Name: 단축코드, dtype: object


In [9]:
def get_previous_posts(base_url):
    posts = []
    headers = {'User-Agent': 'Mozilla/5.0'}
    today = datetime.now()
    date_start = (today - timedelta(days=1)).replace(hour=0, minute=0, second=0)
    # date_end = (today - timedelta(days=1)).replace(hour=23, minute=59, second=59)
    
    page = 0
    while True:
        url = f"{base_url}?&od=T31&category=0&po={page}"
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Failed to fetch page {page}: {response.status_code}")
            break
        
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = soup.select('div.list_item.symph_row')
        
        if not articles:
            print("No more articles found.")
            break
        
        for article in articles:
            # Extract title and link
            title_element = article.select_one('span.subject_fixed')
            date_element = article.select_one('span.timestamp')
            link_element = article.select_one('a.list_subject')
            
            if not title_element or not date_element or not link_element:
                continue
            
            title = title_element.get_text(strip=True)
            date_str = date_element.get_text(strip=True)
            link = f"https://www.clien.net{link_element['href']}"
            
            # Convert date to datetime object
            post_date = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")

            # Stop if the post date is before range
            if post_date < date_start:
                print("No more posts from yesterday.")
                return posts

            if not (date_start <= post_date <= today):
                continue
            
            # Crawling post content
            post_response = requests.get(link, headers=headers)
            if post_response.status_code != 200:
                print(f"Failed to fetch post: {link}")
                continue
            
            post_soup = BeautifulSoup(post_response.text, 'html.parser')
            content_element = post_soup.select_one('div.post_article')
            content = content_element.get_text(strip=True) if content_element else "No content"
            
            posts.append({
                'title': title,
                'date': post_date.strftime("%Y-%m-%d %H:%M"),
                'text': content,
                'link' : link
            })
        page += 1

    return posts


In [14]:
def load_stock_names():
    df_kospi = pd.read_csv('../data/raw_data/kospi_stocks.csv')  # KOSPI 종목명 파일
    df_kosdaq = pd.read_csv('../data/raw_data/kosdaq_stocks.csv')  # KOSDAQ 종목명 파일
    
    # 종목명 컬럼 병합
    stock_names = pd.concat([df_kospi['종목명'], df_kosdaq['종목명']])
    stock_names = stock_names.dropna().str.strip().tolist()
    
    # 긴 이름이 우선 매칭되도록 정렬
    stock_names.sort(key=len, reverse=True)
    
    return stock_names

def extract_stock_keywords(posts):
    # 주식 종목 명 우선 추출
    stock_names = load_stock_names()
    
    # Load KoBERT model, tokenizer
    tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
    model = BertModel.from_pretrained('monologg/kobert')
    
    results = []
    
    for post in posts:
        text = post['title'] + " " + post['text']
        text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text) # 특수 문자 제거 및 전처리
        
        # 종목명 우선 매칭
        match_stock_names = []
        for stock in stock_names:
            if stock in text:
                match_stock_names.append(stock)
                text = text.replace(stock, '')  # 이미 매칭된 종목명은 제거
                
        # KoBERT로 남은 텍스트 토큰화 (KoBERT 최대 길이 = 512)
        split_texts = [text[i:i+512] for i in range(0, len(text), 512)] 
        keywords = []
        
        for chunk in split_texts:
            # 토큰화
            inputs = tokenizer(chunk, return_tensors='pt', truncation=True, padding='max_length', max_length=512)
            
            # KoBERT 모델로 인코딩
            # gradient 계산 비활성화, 훈련 과정이 아니므로 시간 단축을 위해서
            with torch.no_grad():
                outputs = model(**inputs)
            
            # 토큰 ID 배열(['input_ids']) -> 텍스트 변환
            input_ids = inputs['input_ids'][0]
            tokens = tokenizer.convert_ids_to_tokens(input_ids)
            
            # 키워드 필터링: 길이가 1보다 길고, 한국어 문자만 포함
            keywords.extend([token for token in tokens if len(token) > 1 and re.match(r'[가-힣]+', token)])
        
        
        keyword_counts_dict = dict(Counter(match_stock_names + keywords))
        
        results.append({
            'title': post['title'],
            'date': post['date'],
            'text': post['text'],
            'link': post['link'],
            'keywords': keyword_counts_dict
        })
    
    return pd.DataFrame(results)
    

In [15]:
if __name__ == "__main__":
    base_url = "https://www.clien.net/service/board/cm_stock"
    posts = get_previous_posts(base_url)
    
    if posts:
        raw_data_df = extract_stock_keywords(posts)
        print("[Raw data schema]\n")
        print(raw_data_df.head())
    else:
        print("No keywords found.")

No more posts from yesterday.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


[Raw data schema]

                                     title              date  \
0                    오늘 시간외 특징주 요약 - 1월 6일  2025-01-06 19:01   
1                    오늘 장마감 특징주 요약 - 1월 6일  2025-01-06 17:54   
2           오늘 장마감 주도주 / 테마주 순위 요약 - 1월 6일  2025-01-06 15:37   
3                 주식 소각한 다음부터 주가는 상승이 많나요?  2025-01-06 15:25   
4  2024년 연금저축, ISA, IRP 투자 따라하기 (feat 물타님)  2025-01-06 15:14   

                                                text  \
0  오늘 시간외 특징주 요약 - 1월 6일텍스트 버전은 하단에 ..**출처 : 테마랩오...   
1  오늘 장마감 특징주 요약 - 1월 6일텍스트 버전은 하단에 ..**출처 : 테마랩정...   
2  오늘 장마감 주도주 / 테마주 순위 요약 - 1월 6일텍스트 버전은 이미지 하단에 ...   
3  오늘 한 기업이 주식을 소각하는 날 입니다.그러면 향후 주가는 상승할 확률이 높나요...   
4  클리앙에 물타님의 연금저축, ISA, IRP 투자를 보고 따라한 후기 입니다.연금성...   

                                                link  \
0  https://www.clien.net/service/board/cm_stock/1...   
1  https://www.clien.net/service/board/cm_stock/1...   
2  https://www.clien.net/service/board/cm_stock/1...   
3  https://www.clien.net/service/bo